<summary>Table of Contents</summary>

- [1. TimeLLM](#1-timellm)
- [2. TimeLLM](#2-timellm-336)

Results for TimeLLM. The first one is default input length 512, the second one: 336.

In [1]:
import os
import pandas as pd
import subprocess
import time
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

In [2]:
cuda_device = "2"

# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['FR', 'IT']
num_cols = [3, 3]

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# 1. TimeLLM 168

In [3]:
log_dir = f"logs/timellm/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
seq_len = 168
model = "TimeLLM"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_168.log"

# Parameters for tuning,but default
lr = 0.001 # 10^-3 
train_epochs = 20
d_model = 16
d_ff = 64
batch_size = 32

# List to store the results
timellm_results = []

In [4]:
# DE 24 mae:0.09314965456724167
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: FR ===


=== Starting experiments for pred_len: 24 ===

train 86835
val 18651
test 18651
[2024-11-05 17:33:50,243] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-05 17:33:51,233] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-11-05 17:33:51,234] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-05 17:33:51,234] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-11-05 17:33:51,328] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.43, master_port=29500
[2024-11-05 17:33:51,328] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-05 17:33:51,979] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled:

In [5]:
timellm_results

[{'Country': 'FR',
  'Pred_len': 24,
  'MSE': 0.010154812596738338,
  'RMSE': 0.10077109187841415,
  'MAE': 0.05653735250234604},
 {'Country': 'FR',
  'Pred_len': 96,
  'MSE': 0.019628532230854034,
  'RMSE': 0.1401018649339676,
  'MAE': 0.08128844946622849},
 {'Country': 'FR',
  'Pred_len': 168,
  'MSE': 0.02272237464785576,
  'RMSE': 0.15073943138122559,
  'MAE': 0.08884815871715546},
 {'Country': 'IT',
  'Pred_len': 24,
  'MSE': 0.010610697790980339,
  'RMSE': 0.10300824046134949,
  'MAE': 0.05985838919878006},
 {'Country': 'IT',
  'Pred_len': 96,
  'MSE': 0.01913185976445675,
  'RMSE': 0.13831797242164612,
  'MAE': 0.08287986367940903},
 {'Country': 'IT',
  'Pred_len': 168,
  'MSE': 0.020030096173286438,
  'RMSE': 0.14152772724628448,
  'MAE': 0.08731971681118011}]

In [6]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES']
num_cols = [5, 5, 3]

In [7]:
# DE 24 mae:0.09314965456724167
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

train 144725
val 31085
test 31085
[2024-11-06 06:29:26,702] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-06 06:29:27,745] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-11-06 06:29:27,745] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-06 06:29:27,745] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-11-06 06:29:27,829] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.43, master_port=29500
[2024-11-06 06:29:27,829] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-06 06:29:28,975] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled

KeyboardInterrupt: 

In [8]:
timellm_results

[{'Country': 'FR',
  'Pred_len': 24,
  'MSE': 0.010154812596738338,
  'RMSE': 0.10077109187841415,
  'MAE': 0.05653735250234604},
 {'Country': 'FR',
  'Pred_len': 96,
  'MSE': 0.019628532230854034,
  'RMSE': 0.1401018649339676,
  'MAE': 0.08128844946622849},
 {'Country': 'FR',
  'Pred_len': 168,
  'MSE': 0.02272237464785576,
  'RMSE': 0.15073943138122559,
  'MAE': 0.08884815871715546},
 {'Country': 'IT',
  'Pred_len': 24,
  'MSE': 0.010610697790980339,
  'RMSE': 0.10300824046134949,
  'MAE': 0.05985838919878006},
 {'Country': 'IT',
  'Pred_len': 96,
  'MSE': 0.01913185976445675,
  'RMSE': 0.13831797242164612,
  'MAE': 0.08287986367940903},
 {'Country': 'IT',
  'Pred_len': 168,
  'MSE': 0.020030096173286438,
  'RMSE': 0.14152772724628448,
  'MAE': 0.08731971681118011},
 {'Country': 'DE',
  'Pred_len': 24,
  'MSE': 0.022713396698236465,
  'RMSE': 0.15070964395999908,
  'MAE': 0.09314965456724167},
 {'Country': 'DE',
  'Pred_len': 96,
  'MSE': 0.03904155269265175,
  'RMSE': 0.197589352726

In [ ]:
r = [{'Country': 'DE',
  'Pred_len': 24,
  'MSE': 0.022713396698236465,
  'RMSE': 0.15070964395999908,
  'MAE': 0.09314965456724167},
 {'Country': 'DE',
  'Pred_len': 96,
  'MSE': 0.03904155269265175,
  'RMSE': 0.19758935272693634,
  'MAE': 0.1312127262353897},
 {'Country': 'DE',
  'Pred_len': 168,
  'MSE': 0.04313361272215843,
  'RMSE': 0.20768633484840393,
  'MAE': 0.13920451700687408},
 {'Country': 'GB',
  'Pred_len': 24,
  'MSE': 0.025825485587120056,
  'RMSE': 0.16070309281349182,
  'MAE': 0.1029878556728363},
  {'Country': 'GB',
  'Pred_len': 96,
  'MSE': 0.04658650979399681,
  'RMSE': 0.21583908796310425,
  'MAE': 0.14624564349651337},
 {'Country': 'GB',
  'Pred_len': 168,
  'MSE': 0.04751771315932274,
  'RMSE': 0.2179855853319168,
  'MAE': 0.1499413400888443},
{'Country': 'ES',
  'Pred_len': 24,
  'MSE': 0.01031138002872467,
  'RMSE': 0.10154496878385544,
  'MAE': 0.06399503350257874},
 {'Country': 'ES',
  'Pred_len': 96,
  'MSE': 0.01997840777039528,
  'RMSE': 0.14134499430656433,
  'MAE': 0.09172166138887405},
 {'Country': 'ES',
  'Pred_len': 168,
  'MSE': 0.022127792239189148,
  'RMSE': 0.14875413477420807,
  'MAE': 0.09595496952533722},
  {'Country': 'FR',
  'Pred_len': 24,
  'MSE': 0.010154812596738338,
  'RMSE': 0.10077109187841415,
  'MAE': 0.05653735250234604},
 {'Country': 'FR',
  'Pred_len': 96,
  'MSE': 0.019628532230854034,
  'RMSE': 0.1401018649339676,
  'MAE': 0.08128844946622849},
 {'Country': 'FR',
  'Pred_len': 168,
  'MSE': 0.02272237464785576,
  'RMSE': 0.15073943138122559,
  'MAE': 0.08884815871715546},
 {'Country': 'IT',
  'Pred_len': 24,
  'MSE': 0.010610697790980339,
  'RMSE': 0.10300824046134949,
  'MAE': 0.05985838919878006},
 {'Country': 'IT',
  'Pred_len': 96,
  'MSE': 0.01913185976445675,
  'RMSE': 0.13831797242164612,
  'MAE': 0.08287986367940903},
 {'Country': 'IT',
  'Pred_len': 168,
  'MSE': 0.020030096173286438,
  'RMSE': 0.14152772724628448,
  'MAE': 0.08731971681118011}

]

In [5]:
timellm_results = []

In [6]:
# Dynamic variables
pred_lens = [96, 168]
countries = ['GB']
num_cols = [5]

In [7]:
# DE 24 mae:0.09314965456724167
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: GB ===


=== Starting experiments for pred_len: 96 ===

train 144365
val 30725
test 30725
[2024-11-06 16:01:54,342] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-06 16:01:55,406] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-11-06 16:01:55,406] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-06 16:01:55,406] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-11-06 16:01:55,502] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.43, master_port=29500
[2024-11-06 16:01:55,502] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-06 16:01:56,175] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled

In [8]:
timellm_results

[{'Country': 'GB',
  'Pred_len': 96,
  'MSE': 0.04658650979399681,
  'RMSE': 0.21583908796310425,
  'MAE': 0.14624564349651337},
 {'Country': 'GB',
  'Pred_len': 168,
  'MSE': 0.04751771315932274,
  'RMSE': 0.2179855853319168,
  'MAE': 0.1499413400888443}]

In [9]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['ES']
num_cols = [3]

In [10]:
# DE 24 mae:0.09314965456724167
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: ES ===


=== Starting experiments for pred_len: 24 ===

train 86835
val 18651
test 18651
[2024-11-06 18:53:22,545] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-06 18:53:23,543] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-11-06 18:53:23,543] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-06 18:53:23,543] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-11-06 18:53:23,640] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.43, master_port=29500
[2024-11-06 18:53:23,641] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-06 18:53:24,300] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled:

In [11]:
timellm_results

[{'Country': 'GB',
  'Pred_len': 96,
  'MSE': 0.04658650979399681,
  'RMSE': 0.21583908796310425,
  'MAE': 0.14624564349651337},
 {'Country': 'GB',
  'Pred_len': 168,
  'MSE': 0.04751771315932274,
  'RMSE': 0.2179855853319168,
  'MAE': 0.1499413400888443},
 {'Country': 'ES',
  'Pred_len': 24,
  'MSE': 0.01031138002872467,
  'RMSE': 0.10154496878385544,
  'MAE': 0.06399503350257874},
 {'Country': 'ES',
  'Pred_len': 96,
  'MSE': 0.01997840777039528,
  'RMSE': 0.14134499430656433,
  'MAE': 0.09172166138887405},
 {'Country': 'ES',
  'Pred_len': 168,
  'MSE': 0.022127792239189148,
  'RMSE': 0.14875413477420807,
  'MAE': 0.09595496952533722}]

In [13]:
r = [{'Country': 'DE',
  'Pred_len': 24,
  'MSE': 0.022713396698236465,
  'RMSE': 0.15070964395999908,
  'MAE': 0.09314965456724167},
 {'Country': 'DE',
  'Pred_len': 96,
  'MSE': 0.03904155269265175,
  'RMSE': 0.19758935272693634,
  'MAE': 0.1312127262353897},
 {'Country': 'DE',
  'Pred_len': 168,
  'MSE': 0.04313361272215843,
  'RMSE': 0.20768633484840393,
  'MAE': 0.13920451700687408},
 {'Country': 'GB',
  'Pred_len': 24,
  'MSE': 0.025825485587120056,
  'RMSE': 0.16070309281349182,
  'MAE': 0.1029878556728363},
  {'Country': 'GB',
  'Pred_len': 96,
  'MSE': 0.04658650979399681,
  'RMSE': 0.21583908796310425,
  'MAE': 0.14624564349651337},
 {'Country': 'GB',
  'Pred_len': 168,
  'MSE': 0.04751771315932274,
  'RMSE': 0.2179855853319168,
  'MAE': 0.1499413400888443},
{'Country': 'ES',
  'Pred_len': 24,
  'MSE': 0.01031138002872467,
  'RMSE': 0.10154496878385544,
  'MAE': 0.06399503350257874},
 {'Country': 'ES',
  'Pred_len': 96,
  'MSE': 0.01997840777039528,
  'RMSE': 0.14134499430656433,
  'MAE': 0.09172166138887405},
 {'Country': 'ES',
  'Pred_len': 168,
  'MSE': 0.022127792239189148,
  'RMSE': 0.14875413477420807,
  'MAE': 0.09595496952533722},
  {'Country': 'FR',
  'Pred_len': 24,
  'MSE': 0.010154812596738338,
  'RMSE': 0.10077109187841415,
  'MAE': 0.05653735250234604},
 {'Country': 'FR',
  'Pred_len': 96,
  'MSE': 0.019628532230854034,
  'RMSE': 0.1401018649339676,
  'MAE': 0.08128844946622849},
 {'Country': 'FR',
  'Pred_len': 168,
  'MSE': 0.02272237464785576,
  'RMSE': 0.15073943138122559,
  'MAE': 0.08884815871715546},
 {'Country': 'IT',
  'Pred_len': 24,
  'MSE': 0.010610697790980339,
  'RMSE': 0.10300824046134949,
  'MAE': 0.05985838919878006},
 {'Country': 'IT',
  'Pred_len': 96,
  'MSE': 0.01913185976445675,
  'RMSE': 0.13831797242164612,
  'MAE': 0.08287986367940903},
 {'Country': 'IT',
  'Pred_len': 168,
  'MSE': 0.020030096173286438,
  'RMSE': 0.14152772724628448,
  'MAE': 0.08731971681118011}

]

In [14]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/timellm'

if not os.path.exists(path):
    os.makedirs(path)

timellm_df = convert_results_into_df(r, if_loss_fnc=False, itr=1)

# Final DF
timellm_df.columns = pd.MultiIndex.from_product([['TimeLLM/168'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
timellm_df.to_csv(os.path.join(path, 'timellm_168.csv'))
timellm_df.round(4)

Model            TimeLLM/168                
Metrics                  MSE    RMSE     MAE
Country Pred_len                            
DE      24            0.0227  0.1507  0.0931
        96            0.0390  0.1976  0.1312
        168           0.0431  0.2077  0.1392
GB      24            0.0258  0.1607  0.1030
        96            0.0466  0.2158  0.1462
        168           0.0475  0.2180  0.1499
ES      24            0.0103  0.1015  0.0640
        96            0.0200  0.1413  0.0917
        168           0.0221  0.1488  0.0960
FR      24            0.0102  0.1008  0.0565
        96            0.0196  0.1401  0.0813
        168           0.0227  0.1507  0.0888
IT      24            0.0106  0.1030  0.0599
        96            0.0191  0.1383  0.0829
        168           0.0200  0.1415  0.0873

# 2. TimeLLM 336

Sequence length 336.

In [9]:
log_dir = f"logs/timellm/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
model = "TimeLLM"
seq_len = 96

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_96.log"

# Parameters for tuning,but default
lr = 0.001 # 10^-3 
train_epochs = 20
d_model = 16
d_ff = 64
batch_size = 32

In [11]:
# List to store the results
timellm_results = []

start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

train 145085
val 31085
test 31085
[2024-11-05 00:58:33,801] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-05 00:58:34,163] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-11-05 00:58:34,163] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-05 00:58:34,163] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-11-05 00:58:34,251] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.43, master_port=29500
[2024-11-05 00:58:34,251] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-05 00:58:34,957] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled

KeyboardInterrupt: 

In [ ]:
timellm_results

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/timellm'

if not os.path.exists(path):
    os.makedirs(path)

timellm_df = convert_results_into_df(timellm_results, if_loss_fnc=False, itr=1)

# Final DF
timellm_df.columns = pd.MultiIndex.from_product([['TimeLLM/96'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
timellm_df.to_csv(os.path.join(path, 'timellm_96.csv'))
timellm_df.round(4)

Model            TimeLLM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0217  0.1474  0.0920
        96        0.0395  0.1989  0.1324
        168       0.0395  0.1988  0.1348
GB      24        0.0252  0.1588  0.1012
        96        0.0437  0.2091  0.1434
        168       0.0469  0.2166  0.1487
ES      24        0.0108  0.1039  0.0672
        96        0.0198  0.1409  0.0941
        168       0.0217  0.1472  0.0970
FR      24        0.0104  0.1017  0.0586
        96        0.0186  0.1365  0.0824
        168       0.0205  0.1430  0.0878
IT      24        0.0104  0.1022  0.0607
        96        0.0188  0.1370  0.0839
        168       0.0204  0.1428  0.0892

In [ ]:
# List to store the results
timellm_results = []

start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)
